In [ ]:
import os
data_folder = "../input/uw-madison-gi-tract-image-segmentation/" if os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "") else "./data/"

# List all imports below
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scv_utility import *
from unet import *
import torch

np.random.seed(0)
torch.manual_seed(0)
pd.set_option("display.width", 120)

In [ ]:
# Load small train and test datasets with only stomach labels
labels = pd.read_csv(data_folder + "train.csv", converters={"id": str, "class": str, "segmentation": str})
print(f"Classes in train set: {labels['class'].unique()}")

train_cases = ["case2_", "case7_", "case15_", "case20_", "case22_", "case24_", "case29_", "case30_", "case32_", "case123_"]
test_cases = ["case156_", "case154_", "case149_"]
train_labels = labels[labels["id"].str.contains("|".join(train_cases)) & (labels["class"] == "stomach")]
test_labels = labels[labels["id"].str.contains("|".join(test_cases)) & (labels["class"] == "stomach")]
print(f"Number of train images: {len(train_labels['id'])}, test images: {len(test_labels['id'])}")

In [ ]:
# Analyze train and test dataset to assure they have the same resolution
for sample_id in np.concatenate((train_labels["id"].unique(), test_labels["id"].unique())):
    try:
        sample_image, sample_image_res, sample_pixel_size = get_image_data_from_id(sample_id, data_folder)
        #print(f"Image shape: {sample_image.shape}, reported resolution: {sample_image_res}, reported pixel size: {sample_pixel_size}")
        assert sample_image_res == (266, 266) and sample_pixel_size == 1.50, "Incorrect resolution or pixel size"
    except Exception as e:
        print(f"Exception {e} while reading image {sample_id}")
print("Dataset analysis successfull")

In [ ]:

train_data = MRIDataset(data_folder, train_labels)
print(len(train_data))
sample_image, sample_mask = train_data.__getitem__(66)

plt.imshow(sample_image)
plt.imshow(sample_mask, cmap="jet", alpha=0.3)
plt.show()

In [12]:
# CREDITS for a big portion of the training loop: CS4240 DL assignment 3
def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

In [14]:
# in_channels = 1 # Black-white images in MNIST digits
# hidden_channels = [5, 6]
# out_features = 10 

# train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_data, batch_size=1000)

# Training parameters
learning_rate = 0.01
epochs = 3 

# Initialize network
net = UNet()
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate)

# TODO: A reasonable loss function !!!!!!!!!!!!
criterion = nn.CrossEntropyLoss()

# Define list to store losses and performances of each iteration
train_losses = []
train_accs = []
test_accs = []

# Try using gpu instead of cpu
device = try_gpu()

for epoch in range(epochs):

    # Network in training mode and to device
    net.train()
    net.to(device)

    # Training loop
    for i, (x_batch, y_batch) in enumerate(train_loader):

        # Set to same device
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Perform forward pass
        y_pred = net(x_batch)

        # Compute the loss
        loss = criterion(y_pred, y_batch)
        train_losses.append(loss)
        
        # Backward computation and update
        loss.backward()
        optimizer.step()

    # Compute train and test error
#     train_acc = 100*evaluate_accuracy(train_loader, net.to('cpu'))
#     test_acc = 100*evaluate_accuracy(test_loader, net.to('cpu'))
    
    # Development of performance
#     train_accs.append(train_acc)
#     test_accs.append(test_acc)

    # Print performance
    print('Epoch: {:.0f}'.format(epoch+1))
#     print('Accuracy of train set: {:.00f}%'.format(train_acc))
#     print('Accuracy of test set: {:.00f}%'.format(test_acc))
    print('')

NameError: name 'train_loader' is not defined